**Importation**

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model

import itertools
import matplotlib.pyplot as plt
%matplotlib inline

**Data preparation**

In [2]:
train_path  = '../../data/sign_dataset/train'
valid_path  = '../../data/sign_dataset/valid'
test_path  = '../../data/sign_dataset/test'

In [3]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, 
                                                         target_size=(224,224), 
                                                         batch_size=10)

valid_batches = ImageDataGenerator().flow_from_directory(valid_path,
                                                         target_size=(224,224), 
                                                         batch_size=30)

test_batches = ImageDataGenerator().flow_from_directory(test_path, 
                                                        target_size=(224,224), 
                                                        batch_size=50, 
                                                        shuffle=False)

Found 1712 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 51 images belonging to 10 classes.


**Chargement VGG16 pre-trained on ImageNet dataset**

In [4]:
base_model = vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3), pooling='avg')
base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

**Ajout Classifier**

In [5]:
last_layer = base_model.get_layer('block5_pool')

last_output = last_layer.output
x = Flatten()(last_output)
x = Dense(64, activation='relu', name='FC_2')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(10, activation='softmax', name='softmax')(x)

new_model = Model(inputs=base_model.input, outputs=x)

new_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ FC_2 (Dense)                         │ (None, 64)                  │       1,605,696 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 16,321,290 (62.26 MB)

 Trainable params: 16,321,162 (62.26 MB)

 Non-trainable params: 128 (512.00 B)

In [6]:
new_model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

**Accuracy avant entrainement du classifier**

In [7]:
test_loss, test_accuracy = new_model.evaluate(test_batches, steps=test_batches.samples // test_batches.batch_size, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

C:\Users\PaulE\DeepLearning\local\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.1200 - loss: 26.3678
Test Loss: 26.3677978515625
Test Accuracy: 0.11999999731779099


**Entrainement du modele**

In [8]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='signlanguage.model.keras', save_best_only=True)

history = new_model.fit(train_batches, steps_per_epoch=18,
                   validation_data=valid_batches, validation_steps=3, epochs=20, verbose=1, callbacks=[checkpointer])

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 72s 4s/step - accuracy: 0.1197 - loss: 2.4975 - val_accuracy: 0.1778 - val_loss: 19.4089
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 67s 4s/step - accuracy: 0.2819 - loss: 2.0577 - val_accuracy: 0.2333 - val_loss: 21.2781
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 69s 4s/step - accuracy: 0.4530 - loss: 1.5977 - val_accuracy: 0.2556 - val_loss: 10.2737
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 70s 4s/step - accuracy: 0.5417 - loss: 1.3590 - val_accuracy: 0.2333 - val_loss: 4.7684
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 74s 4s/step - accuracy: 0.6055 - loss: 1.1634 - val_accuracy: 0.6222 - val_loss: 1.4246
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 70s 4s/step - accuracy: 0.6932 - loss: 1.1542 - val_accuracy: 0.3222 - val_loss: 3.0165
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 70s 4s/step - accuracy: 0.7334 - loss: 1.0081 - val_accuracy: 0.5111 - val_loss: 1.2756
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 67s 4s/step - accuracy: 0.7739 - loss: 1.0069 - val_accuracy: 0.4889 - val_lo

C:\Users\PaulE\DeepLearning\local\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


18/18 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.7696 - loss: 0.9862 - val_accuracy: 0.7444 - val_loss: 0.7838
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 72s 4s/step - accuracy: 0.8234 - loss: 0.9322 - val_accuracy: 0.5444 - val_loss: 1.1737
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 68s 4s/step - accuracy: 0.8700 - loss: 0.9818 - val_accuracy: 0.5444 - val_loss: 1.4100
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 72s 4s/step - accuracy: 0.7817 - loss: 0.9575 - val_accuracy: 0.7556 - val_loss: 0.9100
Epoch 14/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 73s 4s/step - accuracy: 0.8703 - loss: 0.8040 - val_accuracy: 0.6556 - val_loss: 0.9924
Epoch 15/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 68s 4s/step - accuracy: 0.8207 - loss: 0.8570 - val_accuracy: 0.5111 - val_loss: 1.3876
Epoch 16/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 67s 4s/step - accuracy: 0.8769 - loss: 0.7252 - val_accuracy: 0.6000 - val_loss: 1.3793
Epoch 17/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.9376 - loss: 0.6459 - val_accuracy: 0.7111 - val_loss: 1.0

In [9]:
test_loss, test_accuracy = new_model.evaluate(test_batches, steps=test_batches.samples // test_batches.batch_size, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.8000 - loss: 0.7196
Test Loss: 0.7196117639541626
Test Accuracy: 0.800000011920929


**Rendre les dernières couches du VGG16 entrainable**

In [ ]:
for layer in base_model.layers[:-5]:
    layer.trainable = False
for layer in base_model.layers[-5:]:
    layer.trainable = True
base_model.summary()

**Ajout du classifier**

In [ ]:
last_layer = base_model.get_layer('block5_pool')
last_output = last_layer.output

x = Flatten()(last_output)
x = Dense(64, activation='relu', name='FC_2')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(10, activation='softmax', name='softmax')(x)

new_model_2 = Model(inputs=base_model.input, outputs=x)

new_model_2.summary()

In [ ]:
new_model_2.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

**Accuracy avant entrainment**

In [ ]:
test_loss, test_accuracy = new_model_2.evaluate(test_batches, steps=test_batches.samples // test_batches.batch_size, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer2 = ModelCheckpoint(filepath='signlanguage.lastfeatures_.model.keras', save_best_only=True)

history2 = new_model_2.fit(train_batches, steps_per_epoch=18,
                   validation_data=valid_batches, validation_steps=3, epochs=20, verbose=1, callbacks=[checkpointer2])

In [ ]:
test_loss, test_accuracy = new_model_2.evaluate(test_batches, steps=test_batches.samples // test_batches.batch_size, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.show()